# COGS 108 - Data Checkpoint

# Names

- Jake Heinlein
- Nathan Tripp
- Naomi Chin
- Leo Friedman
- Dante Tanjuatco

<a id='research_question'></a>
# Research Question

*Is the combination of an mlb players age and batting performance, measured by batting average and on base plus slugging percentage, indicative of their yearly salary, and if so, can an algorithm predict a players future contract based on these factors?*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

The first data set looks at MLB players stats by year, which we are using to look at players age, batting average, and on base plus slugging percentage. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
## YOUR CODE HERE

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION